In [3]:
import requests
import os
from openai import OpenAI
import re
from tqdm import tqdm
import pandas as pd
import csv

In [4]:
api_key = os.getenv("OLLAMA_API_KEY")
api_key2 = os.getenv("OLLAMA_API_KEY2")

In [5]:
PROCESSED_QUESTIONS_PATH = "questions_hyde.csv"

questions_df = pd.read_csv('questions_clean.csv')
questions_df.set_index('q_id', inplace=True)

In [6]:
def get_answer_cloud(prompt, api_key):
    data = {
        "model": "qwen3-vl:235b-cloud",
        "prompt": prompt,
        "stream": False,
        "options": {
            "num_predict": 3000
        }
    }

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    url = "https://ollama.com/api/generate"

    response = requests.post(url, json=data, headers=headers)

    return response

In [7]:
def get_answer(response):
        for item in response.output:
            if item.type == 'message':
                if hasattr(item, 'content') and item.content:
                    for content in item.content:
                        if hasattr(content, 'text') and content.text:
                            return content.text

In [8]:
def get_answer_llm(user_prompt):
    system_prompt = f"""На основе вопроса сгенерируй краткий информативный ответ, который должен по-хорошему дать rag система. Ничего кроме краткого ответа на вопрос не нужно, только ответ. Отвечай как чат-бот поддержки альфа-банка.
    Вопрос: {user_prompt}"""
    try:
        response = get_answer_cloud(system_prompt, api_key)
        answer = response.json()["response"]
        return answer
    except:
        ...
    try:
        response = get_answer_cloud(system_prompt, api_key2)
        answer = response.json()["response"]
        return answer
    except:
        ...
    try:
        client = OpenAI(
            base_url="http://127.0.0.1:1234/v1",
            api_key="not-needed"
        )

        response = client.responses.create(
            model="google/gemma-3-27b",
            input=system_prompt,
            max_output_tokens=200,
        )

        return re.sub(r'\n+$', '', get_answer(response))
    except:
        return "жопа"

In [ ]:
if os.path.exists(PROCESSED_QUESTIONS_PATH):
    processed_df = pd.read_csv(PROCESSED_QUESTIONS_PATH)
    processed_ids = set(processed_df['q_id'].values)
else:
    with open(PROCESSED_QUESTIONS_PATH, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['q_id', 'processed_query', 'hypothetical_answer'])
    processed_ids = set()

unprocessed_questions = [(q_id, row) for q_id, row in questions_df.iterrows() 
                        if q_id not in processed_ids]

for q_id, row in tqdm(unprocessed_questions, total=len(unprocessed_questions)):
    query = row['query']

    text = query.lower()
    text = re.sub(r'\+\d[\d\s\-\(\)]+\d', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'©.*$', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^\w\s\.\,\!\?]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    if len(text) == 0:
        text = query.lower()
    
    hypothetical_answer = get_answer_llm(query)
    if hypothetical_answer == "жопа":
        print("жопа")
        break
    with open(PROCESSED_QUESTIONS_PATH, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([q_id, text, hypothetical_answer])

 20%|██        | 1369/6830 [10:50:57<74:02:05, 48.81s/it] 